In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/



drive  sample_data


In [4]:
import zipfile

dataset_path = "/content/drive/MyDrive/CSE465/Mango"  # Make sure this matches the exact file name

print("✅ Dataset extracted successfully!")


✅ Dataset extracted successfully!


**DATASET AUGMENTATION**

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Set directories
original_data_dir = "/content/drive/MyDrive/CSE465/Mango"  # Folder with original images
augmented_data_dir = "/content/augmented_dataset"  # Folder for augmented images
os.makedirs(augmented_data_dir, exist_ok=True)

# Create an ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load original images and apply augmentation
batch_size = 32
num_augmented_images = 0

for class_name in os.listdir(original_data_dir):  # Loop through each category
    class_path = os.path.join(original_data_dir, class_name)
    save_path = os.path.join(augmented_data_dir, class_name)
    os.makedirs(save_path, exist_ok=True)

    for image in os.listdir(class_path):  # Loop through images
        img_path = os.path.join(class_path, image)
        img = tf.keras.preprocessing.image.load_img(img_path)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = img_array.reshape((1,) + img_array.shape)

        # Generate 2 augmented images per original
        i = 0
        for batch in datagen.flow(img_array, batch_size=1, save_to_dir=save_path, save_prefix="aug", save_format="jpg"):
            i += 1
            num_augmented_images += 1
            if i >= 2:  # Change this number to control how many new images per original
                break

print(f"✅ Data augmentation completed! Added {num_augmented_images} images.")


✅ Data augmentation completed! Added 8000 images.


In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import StratifiedKFold
import numpy as np
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
augmented_data_dir = '/content/augmented_dataset'  # Folder where augmented images are stored

# Get the list of all image files and corresponding labels
image_paths = []
image_labels = []

# Traverse the directory to get paths and labels
for label, class_name in enumerate(os.listdir(augmented_data_dir)):
    class_path = os.path.join(augmented_data_dir, class_name)
    if os.path.isdir(class_path):
        for image_name in os.listdir(class_path):
            image_paths.append(os.path.join(class_path, image_name))
            image_labels.append(class_name)  # Use class_name (string) as the label

# Convert to numpy arrays
image_paths = np.array(image_paths)
image_labels = np.array(image_labels)

# Create a DataFrame
df = pd.DataFrame({
    'image_path': image_paths,
    'label': image_labels
})

# Initialize ImageDataGenerator for rescaling
datagen = ImageDataGenerator(rescale=1./255)

# Initialize ResNet50 model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base layers

# Create the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(len(os.listdir(augmented_data_dir)), activation='softmax')  # Adjust the number of classes dynamically
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up 5-fold cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True)

accuracies = []
for train_index, val_index in kfold.split(df['image_path'], df['label']):
    # Split the data into training and validation sets based on indices
    train_df = df.iloc[train_index]
    val_df = df.iloc[val_index]

    # Create new ImageDataGenerators for the current fold
    train_generator = datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col='image_path',
        y_col='label',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=True
    )

    val_generator = datagen.flow_from_dataframe(
        dataframe=val_df,
        x_col='image_path',
        y_col='label',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

    # Train the model on


Found 13672 validated image filenames belonging to 8 classes.
Found 3418 validated image filenames belonging to 8 classes.
Found 13672 validated image filenames belonging to 8 classes.
Found 3418 validated image filenames belonging to 8 classes.
Found 13672 validated image filenames belonging to 8 classes.
Found 3418 validated image filenames belonging to 8 classes.
Found 13672 validated image filenames belonging to 8 classes.
Found 3418 validated image filenames belonging to 8 classes.
Found 13672 validated image filenames belonging to 8 classes.
Found 3418 validated image filenames belonging to 8 classes.


In [10]:
# Save the trained model
model.save("/content/drive/MyDrive/CSE465/model.h5")  # This will save the model to the specified path


In [11]:
from google.colab import files
files.download('my_model.h5')

FileNotFoundError: Cannot find file: my_model.h5

In [12]:
!cp /content/drive/MyDrive/CSE465/model.h5 model.h5

from google.colab import files
files.download("model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Initialize image generators (NO AUGMENTATION here, just rescaling)
datagen = ImageDataGenerator(rescale=1./255)

# Number of classes (adjust this if needed)
num_classes = df['label'].nunique()

# One-hot encode labels
df['label_encoded'] = pd.Categorical(df['label']).codes

# Initialize Stratified K-Fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store metrics
results = []

# Start cross-validation
for fold, (train_idx, val_idx) in enumerate(kfold.split(df['image_path'], df['label_encoded']), 1):
    print(f"\n🔁 Training Fold {fold}/5")

    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]

    train_gen = datagen.flow_from_dataframe(
        train_df,
        x_col='image_path',
        y_col='label',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=True
    )

    val_gen = datagen.flow_from_dataframe(
        val_df,
        x_col='image_path',
        y_col='label',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

    # Define simple CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train model
    model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=5,
        verbose=1
    )

    # Predict on validation data
    val_gen.reset()
    y_true = val_df['label_encoded'].values
    y_pred_probs = model.predict(val_gen, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Calculate metrics
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    print(f"Fold {fold} - Acc: {acc:.4f}, Prec: {precision:.4f}, Rec: {recall:.4f}, F1: {f1:.4f}")

    results.append({
        'Fold': fold,
        'Accuracy': acc,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })

# Convert to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print("\n📊 Final Results Table:\n")
print(results_df)



🔁 Training Fold 1/5
Found 8913 validated image filenames belonging to 8 classes.
Found 2229 validated image filenames belonging to 8 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 868s 3s/step - accuracy: 0.2656 - loss: 3.9438 - val_accuracy: 0.6729 - val_loss: 0.9365
Epoch 2/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 848s 3s/step - accuracy: 0.6686 - loss: 0.9084 - val_accuracy: 0.8484 - val_loss: 0.4469
Epoch 3/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 826s 3s/step - accuracy: 0.8027 - loss: 0.5777 - val_accuracy: 0.8856 - val_loss: 0.3490
Epoch 4/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 812s 3s/step - accuracy: 0.8546 - loss: 0.4279 - val_accuracy: 0.8771 - val_loss: 0.3622
Epoch 5/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 851s 3s/step - accuracy: 0.8972 - loss: 0.3144 - val_accuracy: 0.8919 - val_loss: 0.3118
Fold 1 - Acc: 0.8919, Prec: 0.8991, Rec: 0.8919, F1: 0.8934

🔁 Training Fold 2/5
Found 8913 validated image filenames belonging to 8 classes.
Found 2229 validated image filenames belonging to 8 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


...

279/279 ━━━━━━━━━━━━━━━━━━━━ 797s 3s/step - accuracy: 0.4047 - loss: 2.0193 - val_accuracy: 0.7177 - val_loss: 0.7245
Epoch 2/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 806s 3s/step - accuracy: 0.7530 - loss: 0.6764 - val_accuracy: 0.8582 - val_loss: 0.4060
Epoch 3/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 809s 3s/step - accuracy: 0.8400 - loss: 0.4474 - val_accuracy: 0.8900 - val_loss: 0.3330
Epoch 4/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 804s 3s/step - accuracy: 0.8908 - loss: 0.3290 - val_accuracy: 0.9116 - val_loss: 0.2590
Epoch 5/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 802s 3s/step - accuracy: 0.9166 - loss: 0.2351 - val_accuracy: 0.9098 - val_loss: 0.2654
Fold 4 - Acc: 0.9098, Prec: 0.9124, Rec: 0.9098, F1: 0.9106

🔁 Training Fold 5/5
Found 8914 validated image filenames belonging to 8 classes.
Found 2228 validated image filenames belonging to 8 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


279/279 ━━━━━━━━━━━━━━━━━━━━ 800s 3s/step - accuracy: 0.3770 - loss: 2.3767 - val_accuracy: 0.7706 - val_loss: 0.5918
Epoch 2/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 797s 3s/step - accuracy: 0.7638 - loss: 0.6810 - val_accuracy: 0.8025 - val_loss: 0.4991
Epoch 3/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 820s 3s/step - accuracy: 0.8351 - loss: 0.4575 - val_accuracy: 0.8694 - val_loss: 0.3581
Epoch 4/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 794s 3s/step - accuracy: 0.8793 - loss: 0.3651 - val_accuracy: 0.9080 - val_loss: 0.2859
Epoch 5/5
279/279 ━━━━━━━━━━━━━━━━━━━━ 830s 3s/step - accuracy: 0.9137 - loss: 0.2589 - val_accuracy: 0.8707 - val_loss: 0.4462
Fold 5 - Acc: 0.8707, Prec: 0.8803, Rec: 0.8707, F1: 0.8691

📊 Final Results Table:

   Fold  Accuracy  Precision    Recall  F1 Score
0     1  0.891880   0.899132  0.891880  0.893413
1     2  0.879767   0.883744  0.879767  0.880295
2     3  0.897217   0.899293  0.897217  0.896596
3     4  0.909785   0.912407  0.909785  0.910626
4     5  0.870736   0.880300  0.870736  

In [ ]:
# Step 6: Test Script

import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the trained model
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/CSE465/model.h5')  # Adjust the path if necessary

# Test the model on new images
test_image_path = "/content/drive/MyDrive/CSE465/Mango/Anthracnose/20211008_124249 (Custom).jpg"  # Path to the test image (replace with your test image path)
test_image = image.load_img(test_image_path, target_size=(224, 224))  # Resize to match model input size
test_image_array = image.img_to_array(test_image)  # Convert image to array
test_image_array = np.expand_dims(test_image_array, axis=0)  # Add batch dimension
test_image_array /= 255.0  # Rescale the image (same preprocessing used during training)

# Predict using the model
prediction = loaded_model.predict(test_image_array)  # Get predictions
predicted_class = np.argmax(prediction, axis=1)  # Get the class with the highest probability

# Display the result
print(f"Predicted class: {predicted_class[0]}")  # Print the predicted class


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Predicted class: 1


In [ ]:
!ls /content/augmented_dataset


 Anthracnose	    'Cutting Weevil'  'Gall Midge'  'Powdery Mildew'
'Bacterial Canker'  'Die Back'	       Healthy	    'Sooty Mould'


In [ ]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D


In [ ]:
num_classes = df['label'].nunique()


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


# Load pre-trained ResNet-50 model without the top (classification) layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of ResNet-50 (you can also choose to fine-tune some layers later)
base_model.trainable = False

# Add custom classification layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Optionally unfreeze some layers in ResNet-50 for fine-tuning
for layer in base_model.layers[:100]:
    layer.trainable = False
for layer in base_model.layers[100:]:
    layer.trainable = True


In [ ]:
# Average the results
final_results = results_df.mean()
print("\n📊 Final Average Results:\n")
print(final_results)


NameError: name 'results_df' is not defined

In [ ]:
# Save the model
model.save('final_model.h5')

# To load the model later:
from tensorflow.keras.models import load_model
model = load_model('final_model.h5')


In [ ]:
from google.colab import files
files.download('augmented_dataset.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r augmented_dataset.zip augmented_dataset


  adding: augmented_dataset/ (stored 0%)
  adding: augmented_dataset/Bacterial Canker/ (stored 0%)
  adding: augmented_dataset/Bacterial Canker/aug_0_5284.jpg (deflated 2%)
  adding: augmented_dataset/Bacterial Canker/aug_0_8439.jpg (deflated 2%)
  adding: augmented_dataset/Bacterial Canker/aug_0_7179.jpg (deflated 4%)
  adding: augmented_dataset/Bacterial Canker/aug_0_5802.jpg (deflated 2%)
  adding: augmented_dataset/Bacterial Canker/aug_0_1645.jpg (deflated 2%)
  adding: augmented_dataset/Bacterial Canker/aug_0_5464.jpg (deflated 3%)
  adding: augmented_dataset/Bacterial Canker/aug_0_7415.jpg (deflated 2%)
  adding: augmented_dataset/Bacterial Canker/aug_0_7521.jpg (deflated 2%)
  adding: augmented_dataset/Bacterial Canker/aug_0_4016.jpg (deflated 2%)
  adding: augmented_dataset/Bacterial Canker/aug_0_7270.jpg (deflated 2%)
  adding: augmented_dataset/Bacterial Canker/aug_0_1809.jpg (deflated 2%)
  adding: augmented_dataset/Bacterial Canker/aug_0_8905.jpg (deflated 3%)
  adding: aug